In [22]:
using JuMP
using Gurobi
using DataFrames
using CSV
using LinearAlgebra
using Plots
using Statistics

In [30]:
A = zeros(18,15+11+55+18);
C = zeros(15+11+55+18)

#Shifts of 4h
for j=1:15
    A[j:j+3,j].+=1
    C[j] = 25*4
end

#Shifts of 8h
for j=1:11
    A[j:j+7,j+15].+=1
    C[j+15] = 20*8
end

#Shifts of 1h
for j=1:18
    A[j,j+15+55+11]+=1
    C[j+15+55+11] = 40
end

#Overtime Shifts
n = 0
for k=9:18
    for j=1:(18-k+1)
        A[j:j+k-1,j+15+11+n].+=1
        if k<11
            C[j+15+11+n] = 20*8+30*(k-8)
        else
            C[j+15+11+n] = 20*8+30*2+35*(k-10)
        end
    end
    n+=(18-k+1)
end

In [31]:
D1 = convert(Matrix,CSV.File("HW2_demand.csv"; header=true) |> DataFrame!)[:,2:73];
D = D1[:,1:4:72]

In [26]:
"Build shift allocation model 1"
function allocation_1(D, A, C)
    # extract problem dimensions from distance matrix and verify coherence of input data
    T,J  = size(A)
    M = size(D)[1]
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 30)
    # VARIABLES
    # How many of each shift to use
    @variable(model, x[1:M,1:J] >=0, Int)
    
    #Resource allocation constraint
    @constraint(model, enough_employees[t=1:T, d=1:M],
            sum(A[t, j]*x[d,j]  for j = 1:J) >= D[d, t])
    
    # OBJECTIVE
    @objective(
        model, Min, sum(C[j] * x[d, j] for d=1:M, j=1:J)
    )
    
    optimize!(model)
    
    return model, objective_value(model), x
end

allocation_1

In [27]:
m1, o1, x1 = allocation_1(D, A, C)
X1 = value.(x1)
print("Objective value: ", o1)

Academic license - for non-commercial use only - expires 2021-04-27
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 630 rows, 3465 columns and 28910 nonzeros
Model fingerprint: 0xe14d5bff
Variable types: 0 continuous, 3465 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+02]
Found heuristic solution: objective 1102160.0000
Presolve removed 612 rows and 3366 columns
Presolve time: 0.06s
Presolved: 18 rows, 99 columns, 826 nonzeros
Found heuristic solution: objective 691310.00000
Variable types: 0 continuous, 99 integer (0 binary)

Root relaxation: objective 6.764300e+05, 21 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0    

In [28]:
"Build shelter location model 1"
function allocation_2(D, A, C)
    # extract problem dimensions from distance matrix and verify coherence of input data
    T,J  = size(A)
    M = size(D)[1]
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 300)
    # VARIABLES
    # How many of each shift to use
    @variable(model, x[1:M,1:J] >=0, Int)
    # Match employees with shifts
    @variable(model, y[1:M, 1:90, 1:81], Bin)
    
    #Resource allocation constraint
    @constraint(model, enough_shifts[t=1:T, d=1:M],
            sum(A[t, j]*x[d,j]  for j = 1:J) >= D[d, t])
    
    #No more than max employees available
    @constraint(model, limit_fulltime[d=1:M],
            sum(x[d,j]  for j = 16:81) <= 60)
    @constraint(model, limit_parttime[d=1:M],
            sum(x[d,j]  for j = 1:15) <= 30)
    
    #Don't work more than 2 days in a row
    @constraint(model, no_more_two_days[d=1:33, k=1:90],
            sum(y[d,k,j]+y[d+1,k,j]+y[d+2,k,j] for j=1:81) <=2)
    
    @constraint(model, no_more_one_shift_per_day[d=1:M, k=1:90],
            sum(y[d,k,j] for j=1:81) <=1)
    
    @constraint(model, no_mixing_full_with_part[d=1:M, k=1:60],
            sum(y[d,k,j] for j=1:15) == 0)
    
    @constraint(model, no_mixing_part_with_full[d=1:M, k=61:90],
            sum(y[d,k,j] for j=16:81) == 0)
    
    @constraint(model, meet_shift_employees[d=1:M, j=1:81],
            sum(y[d,k,j]  for k = 1:90) == x[d,j])
    
    # OBJECTIVE
    @objective(
        model, Min, sum(C[j] * x[d, j] for d=1:M, j=1:J)
    )
    
    optimize!(model)
    
    return model, objective_value(model), x, y
end

allocation_2

In [29]:
m2, o2, x2, y2 = allocation_2(D, A, C)
print("Objective value: ", o2)
X2 = value.(x2)

Academic license - for non-commercial use only - expires 2021-04-27
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 12805 rows, 258615 columns and 1367390 nonzeros
Model fingerprint: 0xb26f8256
Variable types: 0 continuous, 258615 integer (255150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 1102160.0000
Presolve removed 3150 rows and 100800 columns
Presolve time: 1.02s
Presolved: 9655 rows, 157815 columns, 779870 nonzeros
Variable types: 0 continuous, 157815 integer (154350 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   24811    7.3621500e+05   5.724048e+04   0.000000e+00      5s
   36065    7.5712250e+05   1.876656e+05   0.000000e+00     10s
   43505    7.7430000e+05   9.895

35×99 Array{Float64,2}:
  0.0  13.0  10.0  0.0  0.0   0.0  0.0  …   0.0  42.0   1.0  17.0  -0.0  -0.0
  0.0   0.0  16.0  0.0  0.0   0.0  0.0      5.0  38.0  -0.0  -0.0  -0.0  -0.0
  0.0   0.0   0.0  0.0  0.0   0.0  0.0      4.0  33.0  28.0  13.0  -0.0  -0.0
 -0.0   6.0   8.0  0.0  0.0   0.0  0.0     10.0  15.0  30.0  -0.0   3.0  -0.0
  0.0   0.0  13.0  0.0  0.0   0.0  0.0      3.0   5.0  -0.0  -0.0   7.0  -0.0
 -0.0   0.0   7.0  0.0  0.0   0.0  0.0  …  14.0  22.0  -0.0   4.0  -0.0  -0.0
  0.0   7.0   0.0  0.0  0.0   0.0  0.0     -0.0  40.0  14.0  -0.0  -0.0  -0.0
  0.0   0.0  13.0  0.0  0.0  -0.0  0.0      0.0  14.0  -0.0  19.0  -0.0  -0.0
  0.0  -0.0   4.0  0.0  0.0   0.0  0.0     31.0  24.0  27.0  40.0   5.0  -0.0
  0.0   2.0   0.0  0.0  0.0   0.0  0.0     32.0  45.0  -0.0  19.0  18.0  -0.0
  0.0   0.0  10.0  0.0  0.0   0.0  0.0  …  10.0  39.0   6.0   0.0  -0.0  -0.0
  0.0   0.0   0.0  0.0  0.0   0.0  0.0     18.0  21.0  31.0   2.0   0.0  -0.0
  0.0   0.0  11.0  0.0  0.0   0.0  0.0  